# Overview

This notebook demostrates the usage of `piper_control`, our lightweight wrapper
of `piper_sdk` for controlling AgileX Piper robots.

The `piper_sdk` API is powerful and quickly maturing, but it's a bit complex and
under-documented, and we found it helpful to define a simple abstraction for
basic I/O.

There are also several sharp bits in `piper_sdk` which can make the robots seem
tempermental, e.g. becoming unresponsive despite repeated calls to
`MotionCtrl_2`, `EnableArm`, `GripperCtrl`, etc. We've bundled our solutions
into `PiperControl` so `reset` and the various move commands perform as one
would expect.

In [1]:
# Enable autoreload (optional, use if modifying piper_control.py)
%load_ext autoreload
%autoreload 2

# CAN Connect
This section demonstrates the use of `piper_connect` to establish the CAN
connection to the robot(s).

It is a python counterpart to AgileX's `can_activate.sh`.

## Sudo access

This cell is only required for granting sudo to `piper_connect.activate`.

`piper_connect` uses `ip` and `ethtool` commands to manage CAN interfaces, and
those tools often require sudo permissions. For this notebook to get sudo
access, you need to run this cell.

### (Optional) passwordless sudo

For deployed robot machines where you don't want to constantly type in your sudo
password, consider adding your user to sudoers file:

```bash
sudo visudo
# Add this line at the end:
<your username> ALL=(ALL) NOPASSWD:ALL
```

### Grant sudo to notebook

In [2]:
import subprocess
import getpass

def sudo_warmup():
    try:
        print("🔐 Sudo setup for Jupyter...")
        pw = getpass.getpass("Enter your password for sudo: ")
        proc = subprocess.run(
            ["sudo", "-S", "echo", "sudo access granted"],
            input=pw + "\n", text=True, check=True, capture_output=True
        )
        print(proc.stdout.strip())
    except subprocess.CalledProcessError as e:
        print("❌ Sudo failed:", e.stderr.strip())

sudo_warmup()


🔐 Sudo setup for Jupyter...
sudo access granted


## Poll and Connect

In [3]:
from piper_control import piper_connect

In [4]:
ports = piper_connect.find_ports()
print(ports)

[('can_right', '1-4.2:1.0'), ('can_left', '1-4.4:1.0'), ('can2', '1-1:1.1')]


In [5]:
piper_connect.activate()
print(piper_connect.active_ports())

['can1', 'can2', 'can0']


# Initialize and Query State

In [15]:
from piper_control import piper_control
from piper_control import piper_init
from piper_control import piper_interface

robot = piper_interface.PiperInterface(can_port="can1")

# Resets the robot and enables the motors and motion controller for the arm.
# This call is necessary to be able to both query state and send commands to the
# robot.
piper_init.reset_arm(
    robot,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
    move_mode=piper_interface.MoveMode.JOINT,
)
# piper_init.reset_gripper(robot)

print(robot.get_joint_positions())
print(robot.get_joint_velocities())
print(robot.get_joint_efforts())
print(robot.get_gripper_state())

[1.4330723710744444, -0.04164355524222223, 0.03417354617111111, -1.1729659570866666, 0.5178740868177778, 0.00918043170888889]
[-0.00019198621444444444, -5.235987666666667e-05, -0.00026179938333333334, -0.0002967059677777778, 0.0, 0.0]
[0.0756, 0.01535625, -0.02480625, -0.41883828, -0.89805828, -0.004792199999999999]
(0.0, 0.0)


In [16]:
# Prints out a higher level status of the robot.
robot.show_status()

Arm Status:
ctrl_mode: CAN_COMMAND
arm_status: NORMAL
mode_feed: JOINT
teach_mode: OFF
motion_status: REACHED_TARGET
trajectory_num: 0
err_code: 0
  Joint 1: limit=OK comms=OK motor=OK
  Joint 2: limit=OK comms=OK motor=OK
  Joint 3: limit=OK comms=OK motor=OK
  Joint 4: limit=OK comms=OK motor=OK
  Joint 5: limit=OK comms=OK motor=OK
  Joint 6: limit=OK comms=OK motor=OK

Gripper Status:
  voltage_too_low     : OK
  motor_overheating   : OK
  driver_overcurrent  : OK
  driver_overheating  : OK
  sensor_status       : OK
  driver_error_status : OK
  driver_enable_status: False
  homing_status       : OK


## Resetting and enabling the robot

In [20]:
# Sometimes, calling `reset()` doesn't work the first time and will throw an
# error. In those instances, you can call reset() again and it should work.
# If it still doesn't work, try turning the robot off and on again.
# And if that _still_ doesn't work, take a look at the CAN connection.

# The robot may physically drop when calling this, so move it to a safe position
# or hold it when calling this.
piper_init.reset_arm(
    robot,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
    move_mode=piper_interface.MoveMode.JOINT,
)
# piper_init.reset_gripper(robot)

## Move a joint

In [ ]:
joint_angles = robot.get_joint_positions()
joint_angles[-2] -= 0.1
print(f"setting joint angles to {joint_angles}")
robot.command_joint_positions(joint_angles)

# Send Gripper Commands

In [ ]:
# Opens the gripper
robot.command_gripper(
    position=piper_interface.GRIPPER_ANGLE_MAX * 0.9,
    effort=piper_interface.GRIPPER_EFFORT_MAX * 0.5,
)

In [ ]:
# Closes the gripper
robot.command_gripper(
    position=piper_interface.GRIPPER_ANGLE_MAX * 0.1,
    effort=piper_interface.GRIPPER_EFFORT_MAX * 0.5,
)

## Move to special positions

In [21]:
# TODO(jscholz): Revisit the rest position.
robot.command_joint_positions(piper_control.REST_POSITION)

## Cartesian Control

In [18]:
robot.set_arm_mode(
    move_mode=piper_interface.MoveMode.POSITION,
    arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
)

ee_pose = robot.get_end_effector_pose()
ee_pose[0] += 0.02  # Move X by 5 cm
print(f"Sending updated pose: {ee_pose}")
robot.command_cartesian_position(ee_pose)

Sending updated pose: [0.094921, -0.028419999999999997, 0.158448, -1.9959584985333334, 0.20146335212111113, -1.5786677347922224]


## Puppeteering

In [ ]:
robot0 = piper_interface.PiperInterface(can_port="can0")
robot1 = piper_interface.PiperInterface(can_port="can1")

In [ ]:

for rbt in [robot0, robot1]:
    piper_init.reset_arm(
        rbt,
        arm_controller=piper_interface.ArmController.POSITION_VELOCITY,
        move_mode=piper_interface.MoveMode.JOINT,
    )
    piper_init.reset_gripper(rbt)


In [ ]:
robot0.command_joint_positions(piper_control.REST_POSITION)

In [ ]:
robot1.command_joint_positions(piper_control.REST_POSITION)

In [ ]:
import time
while True:
  joint_angles = robot1.get_joint_positions()
  print(f"{joint_angles=}")
  time.sleep(0.01)
  robot0.command_joint_positions(joint_angles)